### Import Modules

In [22]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time


sys.path.append("../")
import fitter.data_loader as dl
import fitter.fit_manager as fm
import fitter.fitter as fit
import fitter.special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [46]:
from importlib import reload
reload(dl)
reload(sf)
reload(fit)
reload(fm)

<module 'fitter.fit_manager' from '../fitter/fit_manager.py'>

### Specify fit parameters

In [35]:
p_dict = {
    'collection_name' : 'test', 
    
    'model_info' : {
        'fit_type' : 'xpt',
        'order' : 'nnlo', # 'nlo', 'nnlo', 'nnnlo'
        'F2' : 'FpiFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)

        'include_FV' : True, # max 10
        'exclude' : [], # put LECs here
        'use_bijnens_central_value' : False,
        
        # semi-nnlo corrections
        'include_alpha_s' : False,
        'include_log' : False, 
        'include_log2' : False,
        'include_sunset' : False,
        
        # nnnlo corrections
        'include_latt_n3lo' : False,
    }, 
     #'ma', 'ma-ratio', 'xpt', 'xpt-ratio', or 'poly'; experimental: 'simultaneous', 'ma-old'

    'fast_susnet' : False,
    'use_prior' : True,
    'bias_correct' : True,
    #'include_su2_isospin_corrrection' : False,
    'abbrs' : [u'a06m310L', u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : True,
    
    'save_results' : False,
    'save_pickles' : True,
    'output_name' : 'fit_results',
}

In [7]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()
phys_point_data = data_loader.get_phys_point_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(**p_dict['model_info'], collection_name=p_dict['collection_name'])
    
# Make fit
fit_manager = fm.fit_manager(fit_data, phys_point_data, prior=prior, **p_dict)

if p_dict['make_plots']:
    data_loader.save_plots(
        fit_manager.make_plots(show_error_ellipses=p_dict['show_error_ellipses']),
        output_filename=p_dict['collection_name']+'/fits/'+fit_manager.model
    )
else:
    print(fit_manager)
    
if p_dict['save_results']:
    data_loader.save_fit_info(
        fit_manager.fit_info,
        collection_name=p_dict['collection_name'], 
        save_pickles=p_dict['save_pickles']
    )

    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

Making fit...
Done!
Time (s):  1.047055959701538

Model: xpt_FpiFpi_nnlo_FV

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.1904(27) [1.1957(19)]	(delta_su2 = -0.00410(69))

Parameters:
            L_4    -0.0022 (13)      [   0.0000 (50) ]  
            L_5   -0.00032 (30)      [   0.0000 (50) ]  
            A_k      -1.1 (2.3)      [     0.0 (2.6) ]  
            A_p       2.8 (1.3)      [     0.0 (2.6) ]  *
            A_a      -3.40 (20)      [       0 (100) ]  

Least Square Fit:
  chi2/dof [dof] = 2.2 [17]    Q = 0.0038    logGBF = 57.671

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08*,1e-10,1e-10)    (itns/time = 22/1.0)

Error Budget (relative error):
	disc: 3.12e-06	chiral: 0.000394	pp_input: 0.00152	stat: 0.00165

Total time (s):  2.0277462005615234 



In [8]:
print(fit_manager.fmt_error_budget())

Partial Errors:
                  FK/Fpi
------------------------
          A_a:   0.00000
          A_k:   0.00022
          A_p:   0.00039
          L_4:   0.00011
          L_5:   0.00009
     phys:mpi:   0.00013
      phys:mk:   0.00021
phys:lam2_chi:   0.00179
         stat:   0.00196
------------------------
        total:   0.00271



In [34]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5,
    'L' : np.inf, 
    'a/w0' : 0,
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_a' : 0.1,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


fit_manager.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

NameError: name 'bootstrapper' is not defined